In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import matplotlib.animation as animation

### Voronoi Grid
This uses the same logic as the hexagonal grid, but uses a voronoi grid instead.

In [20]:
# if you ran the Complex Geometry, you don't need to 
# run this cell. You can still run it if you'd like to change
# the structure of the grid, though

grid_resolution = 400 # resolution of the grid across each axis
cell_count = 800 # the number of voronoi cells
voronoi_map, nodes = voronoi_mapping(grid_resolution, cell_count)
adj_dict = gen_adj_dict(voronoi_map, nodes)

In [18]:
# You can tweak these probabilities to create some really cool 
# effects! Try lowering BURNOUT_CHANCE or raising REGROW_ALONE_CHANCE
# to create a self exciting system

FIRE_SPREAD_CHANCE = .6
BURNOUT_CHANCE = .4
REGROW_ALONE_CHANCE =.02
REGROW_NEIGHBOR_CHANCE = .1
FULLY_REGROW_CHANCE = .05

In [23]:
# Define states. These influence the color of each state.
# Larger numbers mean brighter colors

ON = 300 # on is burning 
BURNT = 200 # burnt is not actively recovering
GROWING= 100 # is recovering 
OFF = 0 # can be ignited

random.seed(1379)

node_state_global = [OFF for i in range(cell_count)]
        
def transition(data):
    
    global node_state_global
    node_state = node_state_global.copy()
        
    # this ugly hunk of logic just runs through
    # all the possible states of each node 
    # and updates its state accordingly
    for i in range(cell_count):
        curr_state = node_state_global[i]
        if curr_state == ON:
            for neighbor in adj_dict[i]:
                if node_state[neighbor] == OFF and random.random() < FIRE_SPREAD_CHANCE:
                    node_state[neighbor] = ON
            if random.random() < BURNOUT_CHANCE:
                node_state[i] = BURNT
        elif curr_state == BURNT and random.random() < REGROW_ALONE_CHANCE:
            node_state[i] = GROWING
        elif curr_state == GROWING:
            for neighbor in adj_dict[i]:
                if node_state[neighbor] == BURNT and random.random() < REGROW_NEIGHBOR_CHANCE:
                    node_state[neighbor] = GROWING
            if random.random() < FULLY_REGROW_CHANCE:
                node_state[i] = OFF

        
    # update data
    node_state_global = node_state
    plot_grid = update_voronoi_plot(voronoi_map, node_state)
    mat.set_data(plot_grid)
    return mat

node_state_global[0] = ON
plot_grid = update_voronoi_plot(voronoi_map, node_state_global)
fig, ax = plt.subplots(figsize = (6,6))
mat = ax.imshow(plot_grid, cmap = "summer")
ani = animation.FuncAnimation(fig, transition, interval=50,
                              save_count=50)

plt.close(fig)


from IPython.display import HTML
HTML(ani.to_jshtml())